# [STARTER] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [1]:
import os, json
from datetime import datetime
from typing import List, Dict
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

## Play with Tavily

In [2]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [3]:
# [TODO] Define a query and run
query = "when will construction in piazza venezia be completed"
result = client.search(query)

In [4]:
result

{'query': 'when will construction in piazza venezia be completed',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.saturdaysinrome.com/blog/piazza-venezia-construction/',
   'title': 'The Piazza Venezia Construction Is An Obstacle For Visitors',
   'content': "The current (as of 2023) thinking is that the project will be completed by 2033, but I don't think long term planning is possible with a construction project",
   'score': 0.83932644,
   'raw_content': None},
  {'url': 'https://www.webuildgroup.com/en/discovery/video/project-murales-contemporary-art-metro-c-piazza-venezia-construction-site/',
   'title': 'Project "Murales", contemporary art in the Metro C Piazza Venezia ...',
   'content': "Starting in December 2024, new pieces of art will be featured every four months on the silos of the Metro C's Piazza Venezia construction site, transforming",
   'score': 0.79637206,
   'raw_content': None},
  {'url': 'https://www.webuildgroup.co

## Define Web Search tool

In [5]:
@tool
def web_search(query: str, search_depth: str = "advanced", tavily_client: TavilyClient = client) -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
        tavily_client (TavilyClient): Tavily instance
    """
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

In [6]:
tools = [web_search]

## Define Agents

The first agent should not use tools, just its own knowledge. The second one should have web search tool enabled.

In [8]:
# [TODO] Agent with no tools
simple_agent = Agent(model_name="gpt-4o-mini", instructions="You are a helpful assistant.")

In [9]:
web_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
            "You are a web-aware assistant that can search for update information "
            "For each query, you will search the web for current information using " 
            "Tavily's AI-optimized search and provide a comprehensive answer\n"
            "Always cite your sources and explain any discrepancies found.\n"
            "Be particularly attentive to dates and time-sensitive information."
    ),
    tools=tools
)

In [10]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

Run the Agents and compare them. The following queries are just for reference. Change them as you want.

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [11]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I cannot provide information about events or awards that occurred after my knowledge cutoff date in October 2023. You may want to check a reliable news source or the official Oscars website for the most current information on the 2025 Oscars., tool_calls = None)


In [12]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I cannot provide information about events or awards that occurred after my knowledge cutoff date in October 2023. You may want to check a reliable news source or the official Oscars website for the most current information on the 2025 Oscars.


In [13]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I cannot provide information about events or awards that occurred after my knowledge cutoff date in October 2023. You may want to check a reliable news source or the official Oscars website for the most current information on the 2025 Oscars., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last knowledge update in October 2023, here are some notable trends and developments in AI technology:

1. **Generative AI Advances**: Generative AI models, particularly large la

In [14]:
print(run2.get_final_state()["messages"][-1].content)

As of my last knowledge update in October 2023, here are some notable trends and developments in AI technology:

1. **Generative AI Advances**: Generative AI models, particularly large language models (LLMs) like GPT-4, have continued to evolve, with improvements in their ability to generate coherent and contextually relevant text, as well as in applications such as image and video generation.

2. **AI in Healthcare**: AI applications in healthcare have expanded, with improved diagnostic tools, predictive analytics for patient outcomes, and AI-driven drug discovery processes. Machine learning models are increasingly used to analyze medical images and provide insights for treatment plans.

3. **Ethical AI and Regulation**: There has been a significant focus on the ethical implications of AI, leading to discussions about regulations and frameworks to ensure responsible AI development and deployment. Governments and organizations are working on guidelines to address bias, privacy, and tra

**Web Agent**

In [15]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageFunctionToolCall(id='call_fHLpgdYAN9vzxo0vCKXmqUiB', function=Function(arguments='{"query":"2025 Oscar winner International Movie","search_depth":"basic"}', name='web_s

In [16]:
print(run1.get_final_state()["messages"][-1].content)

The winner of the 2025 Oscar for Best International Feature Film is Brazil's *I'm Still Here*. This film, directed by Walter Salles, is based on a true story set in 1970s Rio de Janeiro during the period of military dictatorship. The film stars Fernanda Torres, who was also nominated for her leading role in the movie. 

In addition to *I'm Still Here* winning in the international category, the film *Anora* took home the award for Best Picture at the same ceremony. Other notable winners included *The Brutalist* for Best Cinematography and Zoe Saldaña for her role in *Emilia Pérez*.

For more details, you can check the sources from NPR [here](https://www.npr.org/2025/03/02/nx-s1-5315313/oscars-2025-im-still-here-brazil-best-international-feature) and BBC [here](https://www.bbc.com/news/articles/c1mngz9zreno).


In [17]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageFunctionToolCall(id='call_fHLpgdYAN9vzxo0vCKXmqUiB', function=Function(arguments='{"query":"2025 Oscar winner International Movie","search_depth":"basic"}', name='web_s

In [18]:
print(run2.get_final_state()["messages"][-1].content)

In 2023, the field of artificial intelligence (AI) experienced significant advancements and breakthroughs across various domains. Here are some of the most notable developments:

1. **Generative AI**: This year marked a breakthrough for generative AI technologies. Tools like Microsoft Copilot, GPT-4, and Claude 2 gained prominence, transforming how users interact with AI for content creation, coding assistance, and more. Generative models have become more sophisticated, allowing for more realistic and contextually aware outputs across various applications.

2. **Healthcare Innovations**: The FDA approved 223 AI-enabled medical devices in 2023, a dramatic increase from just six approvals in 2015. This indicates a growing trust in AI technologies to support healthcare providers and improve patient outcomes.

3. **Natural Language Processing (NLP)**: Advances in NLP are making AI systems better at understanding and generating human language. This improvement is critical for applications l

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow